Here we go from individual bams to chr-separated VCFs with all samples in each VCF (WhatsHap needs separation by chr, so we merge later)

*Note* 
This expects the ref file to bei faidx-indexed and the chromosome names to begin with "NC", the order of which determines then autosomes, x, y, mt as is the case in `GCF_000001635.26_GRCm38.p6_genomic.fna`

In [1]:
# set important data in the config file first!
source("config.R")
source("../../extra-R-functions.R")

v Reading from Status of mouse sequencing.

v Range ''All founder files''.


Attaching package: 'tidytable'


The following objects are masked from 'package:stats':

    dt, filter, lag


The following object is masked from 'package:base':

    %in%




In [2]:
library(naturalsort)

In [3]:
files=list.files(path = bam_dir, pattern="[0-9].bam$", full.names = TRUE)

files

character(0)

In [4]:
jobname="samtools_merge"


In [5]:
indivs=as.data.frame(table(files_df$ID))
for(i in 1:nrow(indivs)){
    #check if all bams that should exist do exist
    filenames=paste0(indivs$Var1[i],".",1:indivs$Freq[i],".bam")

    if(sum(!(filenames%in%basename(files)))==0){
        print(paste(indivs$Var1[i],"files completely present"))
        if(file.exists(paste0(bam_dir,"/",indivs$Var1[i],
                   ".merged.bam"))){
            print(paste(indivs$Var1[i],"already merged, but files still exist? skipping..."))
        }
        cmd=paste0("cd ",bam_dir, " && samtools merge -cf -o ",indivs$Var1[i],
                   ".merged.bam ",paste(filenames, collapse = " "))
        cmd=paste0(cmd," && ",
                  "samtools index ",indivs$Var1[i],
                   ".merged.bam && ",
                  paste(paste0("touch ",filenames,".merged"), collapse = " && "),
                   " && ",
                  "rm -f ",paste(filenames, collapse = " "))
        #print(cmd)
        execute_cmd_sbatch(cmd, mem="16gb", cpu="1", time="short", env=env_mapping_etc, jobname=jobname,
                          activateEnvScript=paste0(Barn_Mice_dir,"activateEnv.sh")) 
        Sys.sleep(1)
    }else{
        print(paste(indivs$Var1[i],"MISSING FILES"))
        }
}

[1] "SW_1 MISSING FILES"
[1] "SW_10 MISSING FILES"
[1] "SW_11 MISSING FILES"
[1] "SW_1133 MISSING FILES"
[1] "SW_1140 MISSING FILES"
[1] "SW_12 MISSING FILES"
[1] "SW_2 MISSING FILES"
[1] "SW_25 MISSING FILES"
[1] "SW_3 MISSING FILES"
[1] "SW_31 MISSING FILES"
[1] "SW_4 MISSING FILES"
[1] "SW_5 MISSING FILES"
[1] "SW_6 MISSING FILES"
[1] "SW_7 MISSING FILES"
[1] "SW_8 MISSING FILES"
[1] "SW_80 MISSING FILES"
[1] "SW_84 MISSING FILES"
[1] "SW_85 MISSING FILES"
[1] "SW_87 MISSING FILES"
[1] "SW_9 MISSING FILES"
[1] "SW_90 MISSING FILES"


In [6]:
while(slurm_check_jobs_still_running(columbia_username,jobname)){
    Sys.sleep(60)
}

In [7]:
files=list.files(path = bam_dir, pattern="merged.bam$", full.names = TRUE)
# this individual is not needed
if(paste0(bam_dir,"/","SW_1140.merged.bam") %in% files){
    file.remove(paste0(bam_dir,"/","SW_1140.merged.bam"))
    files=list.files(path = bam_dir, pattern="merged.bam$", full.names = TRUE)
}

In [8]:
files

[1] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/SW_1.merged.bam"   
 [2] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/SW_10.merged.bam"  
 [3] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/SW_11.merged.bam"  
 [4] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/SW_1133.merged.bam"
 [5] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/SW_12.merged.bam"  
 [6] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/SW_2.merged.bam"   
 [7] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/SW_25.merged.bam"  
 [8] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/SW_3.merged.bam"   
 [9] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/SW_31.merged.bam"  
[10] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/SW_4.merged.bam"   
[11] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/SW_5.merged.bam"   
[12] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/SW_6.merged.bam"   
[13] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/SW_7.merged.bam"   
[14] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/SW_8.merged.bam"   
[15] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/SW_80.merged.bam"  
[16] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/SW_84.merged.bam"  
[17] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/SW_85.merged.bam"  
[18] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/SW_87.merged.bam"  
[19] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/SW_9.merged.bam"   
[20] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/SW_90.merged.bam"

In [9]:
# PED file  with samplename\tM/F
ped=fread("../../XX_Data/FoundersF1.no1140.ped")
females=ped%>%select(Mother)%>% 
           unlist(., use.names=FALSE)
females=c(females,
         ped%>%filter(Sex=="F")%>%select(Individual)%>% 
           unlist(., use.names=FALSE))
females=unique(females)
males=ped%>%select(Father)%>% 
           unlist(., use.names=FALSE)
males=c(males,
         ped%>%filter(Sex=="M")%>%select(Individual)%>% 
           unlist(., use.names=FALSE))
males=unique(males)
fwrite(data.table(ID=c(females,males),Sex=c(rep("F",length(females)),
                                rep("M",length(males)))),
          PED_file<-paste0(bam_dir,"/","PED.tsv"),col.names=FALSE,sep="\t")

In [10]:
chrs=fread(cmd=paste0('grep "^NC" ',ref_file,".fai"))
chrs=select(chrs,V1)%>% 
           unlist(., use.names=FALSE)
chrs=sort(chrs)
chrs=data.table(chrFasta=chrs,
           chrSimple=paste0("chr",c(1:(length(chrs)-3),"X","Y","MT")))
chrs

chrFasta,chrSimple
<chr>,<chr>
NC_000067.6,chr1
NC_000068.7,chr2
NC_000069.6,chr3
NC_000070.6,chr4
NC_000071.6,chr5
NC_000072.6,chr6
NC_000073.6,chr7
NC_000074.6,chr8
NC_000075.6,chr9


In [11]:
jobname="F0_mpileup"

In [12]:
for(i in 1:nrow(chrs)){
    output=paste0("Founders.",chrs$chrSimple[i],".mpileup")
    if(file.exists(paste0(bam_dir,"/",output))){
        next
    }
    cmd=paste0("cd ",bam_dir," && bcftools mpileup --threads 8 -f ", ref_file, " -q 30 -Q 30 ",
          "--skip-indels -a FORMAT/AD,FORMAT/DP,INFO/AD -r ",
          chrs$chrFasta[i], " -Ob -o ",output, " ", paste(basename(files),collapse=" "))
    print(cmd)
    execute_cmd_sbatch(cmd=cmd, mem="16G", cpu="8", time="long", acc=slurm_acc, env=env_mapping_etc, jobname=jobname<-"F0_mpileup",
                      activateEnvScript=paste0(Barn_Mice_dir,"activateEnv.sh"))
    Sys.sleep(1)
}

In [13]:
jobname
while(slurm_check_jobs_still_running(columbia_username,jobname)){
    Sys.sleep(60)
}
files=naturalsort(list.files(path = bam_dir, pattern=".mpileup$", full.names = TRUE))
files

[1] "F0_mpileup"

[1] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr1.mpileup" 
 [2] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr2.mpileup" 
 [3] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr3.mpileup" 
 [4] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr4.mpileup" 
 [5] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr5.mpileup" 
 [6] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr6.mpileup" 
 [7] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr7.mpileup" 
 [8] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr8.mpileup" 
 [9] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr9.mpileup" 
[10] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr10.mpileup"
[11] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr11.mpileup"
[12] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr12.mpileup"
[13] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr13.mpileup"
[14] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr14.mpileup"
[15] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr15.mpileup"
[16] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr16.mpileup"
[17] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr17.mpileup"
[18] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr18.mpileup"
[19] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr19.mpileup"
[20] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chrMT.mpileup"
[21] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chrX.mpileup" 
[22] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chrY.mpileup"

In [14]:
PED_file

[1] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/PED.tsv"

In [15]:
jobname="F0_call"
for(f in files){
    c=getWhich(basename(f), split=".", which=2)

    ploidy_flag="--ploidy 2"
    # -Y and -X for the sex chromosomes
    # 1 for MT
    if(c == "chrY"){
        ploidy_flag="-Y"
    }
    if(c == "chrX"){
        ploidy_flag="-X"
    }
    if(c == "chrMT"){
        ploidy_flag="--ploidy 1"
    }
    
    filename=str_replace(f, fixed(".mpileup"), ".vcf.gz")
    
    cmd=paste0("cd ",bam_dir, " && bcftools call -Oz -o ",filename," ",
              "",ploidy_flag," --threads 8 -a FORMAT/GQ,FORMAT/GP ",
              "-m -v -V indels -S ",PED_file, " ", f, " && bcftools index ",filename, 
               " && bcftools query -l ",filename, " > ",filename,".samples")
    
    print(cmd)
    execute_cmd_sbatch(cmd=cmd, mem="16G", cpu="8", time="short", acc=slurm_acc, env=env_mapping_etc, jobname=jobname,
                      activateEnvScript=paste0(Barn_Mice_dir,"activateEnv.sh"))
    Sys.sleep(1)
}

[1] "cd /moto/ziab/users/jr3950/data/genomes/tmp_founders/bam && bcftools call -Oz -o /moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr1.vcf.gz --ploidy 2 --threads 8 -a FORMAT/GQ,FORMAT/GP -m -v -V indels -S /moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/PED.tsv /moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr1.mpileup && bcftools index /moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr1.vcf.gz && bcftools query -l /moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr1.vcf.gz > /moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr1.vcf.gz.samples"
[1] "sbatch -c 8 --mem=16G --job-name=F0_call -A ziab -t 11:59:00 --wrap '. ~/ColumbiaProjects/Barn_Mice/activateEnv.sh samtools-116; cd /moto/ziab/users/jr3950/data/genomes/tmp_founders/bam && bcftools call -Oz -o /moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr1.vcf.gz --ploidy 2 --threads 8 -a FORMAT/GQ,FORMAT/GP -m -v -V indels -S /moto/

In [16]:
# check each combination of IDs for lowest Mendelian errors
# sanity check
trios=crossing(mother = females, father = males, offspring = c(females, males))
trios=filter(trios, !(offspring%in%paste0("SM_SW_",1:12)), mother %in% paste0("SM_SW_",1:12),
            father %in% paste0("SM_SW_",1:12))
fwrite(trios, paste0(bam_dir,"/","test_trios.txt"), sep=",", col.names = FALSE)




jobname
while(slurm_check_jobs_still_running(columbia_username,jobname)){
    Sys.sleep(60)
}

files=naturalsort(list.files(path = bam_dir, pattern="chr[0-9]*.vcf.gz$", full.names = TRUE))
jobname="Mendel"
for(f in files){
    cmd=paste0("bcftools +mendelian ",f," -T ",paste0(bam_dir,"/","test_trios.txt")," -m c > ",f,".mendel.txt")
    execute_cmd_sbatch(cmd=cmd, mem="16G", cpu="8", time="short", acc=slurm_acc, env=env_mapping_etc, jobname=jobname,
                      activateEnvScript=paste0(Barn_Mice_dir,"activateEnv.sh"))
    Sys.sleep(1)
}
jobname
while(slurm_check_jobs_still_running(columbia_username,jobname)){
    Sys.sleep(60)
}


[1] "F0_call"

[1] "sbatch -c 8 --mem=16G --job-name=Mendel -A ziab -t 11:59:00 --wrap '. ~/ColumbiaProjects/Barn_Mice/activateEnv.sh samtools-116; bcftools +mendelian /moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr1.vcf.gz -T /moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/test_trios.txt -m c > /moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr1.vcf.gz.mendel.txt'"
[1] "Submitted batch job 13460466"
[1] "sbatch -c 8 --mem=16G --job-name=Mendel -A ziab -t 11:59:00 --wrap '. ~/ColumbiaProjects/Barn_Mice/activateEnv.sh samtools-116; bcftools +mendelian /moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr2.vcf.gz -T /moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/test_trios.txt -m c > /moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr2.vcf.gz.mendel.txt'"
[1] "Submitted batch job 13460467"
[1] "sbatch -c 8 --mem=16G --job-name=Mendel -A ziab -t 11:59:00 --wrap '. ~/ColumbiaProjects/Barn_Mice/activateEnv.sh samtools-116; bcftool

[1] "Mendel"

In [18]:

# Read in the data for all chromosomes
mendelian_counts <- lapply(1:19, function(chr) {
  fread(paste0("/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr", chr, ".vcf.gz.mendel.txt"))
}) %>%
  bind_rows()

# Rename the columns
colnames(mendelian_counts) <- c("nOK", "nBad", "nSkipped", "Trio")

mendelian_counts=summarise(group_by(mendelian_counts, 
                                   Trio),
                          nOK=sum(nOK), nBad=sum(nBad), nSkipped=sum(nSkipped))

# Split the Trio column into separate mother, father, and child columns
mendelian_counts <- separate(mendelian_counts, Trio, into = c("mother", "father", "child"), sep = ",")

# Calculate the number of errors per child for each trio
mendelian_counts <- mendelian_counts %>%
  group_by(child) %>%
  mutate(errors_per_child = nBad / (nBad+nOK))

# Identify the trio with the lowest number of errors per child for each child
best_trios <- mendelian_counts %>%
  group_by(child) %>%
  filter(errors_per_child == min(errors_per_child)) %>%
  arrange(errors_per_child)

print(mean(mendelian_counts$errors_per_child))

print(mean(mendelian_counts$errors_per_child[mendelian_counts$child=="SM_SW_1133"]))

merged_data <- merge(ped, best_trios, by.x = "Individual", by.y = "child")

# Print the best trio for each child
print(select(merged_data, Individual, Father, father, Mother, mother, errors_per_child) %>%
  arrange(errors_per_child))

[1] 0.1502124
[1] 0.1684524
# A tidytable: 8 x 6
  Individual Father   father   Mother   mother   errors_per_child
  <chr>      <chr>    <chr>    <chr>    <chr>               <dbl>
1 SM_SW_25   SM_SW_9  SM_SW_9  SM_SW_7  SM_SW_7            0.0240
2 SM_SW_90   SM_SW_3  SM_SW_3  SM_SW_2  SM_SW_2            0.0257
3 SM_SW_84   SM_SW_6  SM_SW_6  SM_SW_5  SM_SW_5            0.0269
4 SM_SW_87   SM_SW_3  SM_SW_3  SM_SW_1  SM_SW_1            0.0271
5 SM_SW_85   SM_SW_6  SM_SW_6  SM_SW_4  SM_SW_4            0.0273
6 SM_SW_80   SM_SW_12 SM_SW_12 SM_SW_10 SM_SW_10           0.0274
7 SM_SW_31   SM_SW_12 SM_SW_12 SM_SW_11 SM_SW_11           0.0299
8 SM_SW_1133 SM_SW_9  SM_SW_9  SM_SW_8  SM_SW_8            0.0647


In [21]:

files=naturalsort(list.files(path = bam_dir, pattern="chr[0-9]*.vcf.gz$", full.names = TRUE))
files
if(length(files)==nrow(chrs)) # all mpileups exist
    {
    print("Files exist, deleting, mpileups...")
    file.remove(list.files(path = bam_dir, pattern=".mpileup$", full.names = TRUE))
}

[1] "F0_call"

[1] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr1.vcf.gz"  
 [2] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr2.vcf.gz"  
 [3] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr3.vcf.gz"  
 [4] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr4.vcf.gz"  
 [5] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr5.vcf.gz"  
 [6] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr6.vcf.gz"  
 [7] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr7.vcf.gz"  
 [8] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr8.vcf.gz"  
 [9] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr9.vcf.gz"  
[10] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr10.vcf.gz" 
[11] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr11.vcf.gz" 
[12] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr12.vcf.gz" 
[13] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr13.vcf.gz" 
[14] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr14.vcf.gz" 
[15] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr15.vcf.gz" 
[16] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr16.vcf.gz" 
[17] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr17.vcf.gz" 
[18] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr18.vcf.gz" 
[19] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chr19.vcf.gz" 
[20] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chrMT.vcf.gz" 
[21] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chrX.vcf.gz"  
[22] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.chrY.vcf.gz"  
[23] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/Founders.merged.vcf.gz"